# Install requirements

In [ ]:
%pip install -r requirements.txt

# Obtain input data

In [ ]:
import os 

from challenge import get_datasets, TEMP_PATH

In [ ]:
conn_string = dbutils.secrets.get(scope="storage-secret-scope", key="storage_conn_string")

get_datasets(conn_string)

In [ ]:
ads_df = spark.read.json(f"file://{TEMP_PATH}/ads.json")
views_df = spark.read.json(f"file://{TEMP_PATH}/views.json")


In [ ]:
ads_df.show()

# Flatten and clean datasets

In [ ]:
from challenge import flatten, remove_underscores_from_column

In [ ]:
ads_df = flatten(ads_df)
views_df = flatten(views_df)
ads_df.show()

In [ ]:
ads_df = remove_underscores_from_column(ads_df,"attributes__exteriorColor")
ads_df.show()
